In [6]:
## add missing exchange reactions to gapfilled models
# This script adds missing exchange reactions for specific metabolites in gapfilled models.
# It checks if the metabolites are present in the model and adds exchange reactions for them if they are not.

from cobra import Reaction, Metabolite
import cobra
import os

os.environ['GRB_LICENSE_FILE'] = '../content/licenses/gurobi.lic'

def makeMissingReaction(model, met_in, met_out):
    new_reaction_1 = Reaction('EX_' + met_out.id)
    new_reaction_1.name = met_out.name + ' exchange'
    new_reaction_1.lower_bound = -1000
    new_reaction_1.upper_bound = 1000
    new_reaction_1.add_metabolites({met_out: -1.0})

    new_reaction_2 = Reaction('TR_' + met_in.id)
    new_reaction_2.name = met_in.name + ' transport'
    new_reaction_2.lower_bound = -1000
    new_reaction_2.upper_bound = 1000
    new_reaction_2.add_metabolites({met_out: -1.0, met_in: 1.0})

    model.add_reactions([new_reaction_1, new_reaction_2])
    return model

def fix_exchange_reactions(model_in, list_of_metabolites):
    for met in list_of_metabolites:
        if model_in.metabolites.has_id(met):
            met_in = model_in.metabolites.get_by_id(met)
            met_out = Metabolite(met_in.id.replace("_c0", "_e0"),
                                 formula = met_in.formula,
                                 name= met_in.name.replace("_c0", "_e0"),
                                 compartment='e0')
            makeMissingReaction(model_in, met_in, met_out)
            print(f"Added exchange reaction for {met} in {model_in.id}")
    return model_in.copy()

gapfilled_dir = "../models/Gapfilled_models"
output_dir = "../models/fixed_model_V2"

os.makedirs(output_dir, exist_ok=True)

model_files = sorted([f for f in os.listdir(gapfilled_dir) if f.endswith(".xml")])

list_of_metabolites = [
    "cpd00211_c0",  # Butyrate
    "cpd00047_c0",  # Formate
    "cpd00029_c0",  # Acetate
    "cpd00141_c0",  # Propionate
    "cpd00159_c0",  # L-Lactate
    "cpd00221_c0",  # D-Lactate
]

for model_file in model_files[12:13]:
    model_path = os.path.join(gapfilled_dir, model_file)
    model = cobra.io.read_sbml_model(model_path)
    model = fix_exchange_reactions(model, list_of_metabolites)

    output_path = os.path.join(output_dir, model_file)
    cobra.io.write_sbml_model(model, output_path)


Added exchange reaction for cpd00211_c0 in MAG013_gapfilled
Added exchange reaction for cpd00047_c0 in MAG013_gapfilled
Added exchange reaction for cpd00029_c0 in MAG013_gapfilled
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0ghe5_tv.lp
Reading time = 0.00 seconds
: 853 rows, 1670 columns, 7728 nonzeros


In [ ]:
# This script extracts exchange reactions from gapfilled models and saves them to a unified file.

import os
import cobra
gapfilled_dir = "../models/fixed_models"  # folder containing gapfilled models
model_files = sorted([f for f in os.listdir(gapfilled_dir) if f.endswith(".xml")])

def extract_exchange_reactions(model):
    """
    Return a dictionary of exchange reactions from the model
    that have negative lower_bound (< 0), i.e., import direction.

    Format: {rxn_id: [met_id, met_name, met_formula]}
    """
    exchange_reactions = {}
    for reaction in model.reactions:
        # Common practice: exchange rxns often start with "EX_",
        # but you can adapt this filter if your exchange IDs differ
        if reaction.id.startswith("EX_") and (reaction.lower_bound < 0):
            # Typically there's only 1 metabolite in an exchange reaction,
            # so we grab the first one
            if len(reaction.metabolites) == 1:
                met = list(reaction.metabolites.keys())[0]
                exchange_reactions[reaction.id] = [met.id, met.name, met.formula]

    return exchange_reactions

# 1) Collect exchange reactions from each gapfilled model
all_exchange_dict = {}  # key=rxn_id, value=[met_id, met_name, met_formula]

for mf in model_files:
    model_path = os.path.join(gapfilled_dir, mf)
    print(f"Extracting exchange rxns from {mf}...")
    try:
        model = cobra.io.read_sbml_model(model_path)
    except Exception as e:
        print(f"[Error] Cannot load {mf}: {e}")
        continue

    ex_rxns = extract_exchange_reactions(model)

    # Unify them
    for rxn_id, info_list in ex_rxns.items():
        if rxn_id not in all_exchange_dict:
            all_exchange_dict[rxn_id] = info_list

# 2) Write the unified exchange reactions to a file
medium_file = "../models/group_medium_exchanges.tsv"
with open(medium_file, "w") as f:
    f.write("rxn_id\tmet_id\tmet_name\tmet_formula\n")
    for rxn_id, info in all_exchange_dict.items():
        met_id, met_name, met_form = info
        f.write(f"{rxn_id}\t{met_id}\t{met_name}\t{met_form}\n")

print(f"\n[Unified exchange reactions saved to: {medium_file}]")
print(f"Total unique exchange reactions: {len(all_exchange_dict)}")


Extracting exchange rxns from MAG001.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG002.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG003.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG004.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG005.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG006.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG007.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG008.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG009.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG010.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG011.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG012.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG013.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG014.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG015.sbml_gapfilled_noO2.xml...
Extracting exchange rxns from MAG016.sbm

## check block reaction

In [ ]:
from cobra.io import read_sbml_model
import os
os.environ['GRB_LICENSE_FILE'] = '../content/licenses/gurobi.lic'


In [7]:
import os
import pandas as pd
from cobra.io import read_sbml_model
from cobra.flux_analysis import flux_variability_analysis

models_dir = "../models/fixed_models"

target_reactions = {
    "Butyrate": "EX_cpd00211_e0",
    "Propionate": "EX_cpd00141_e0"
}

results = []

for file in sorted(os.listdir(models_dir)):
    if file.endswith(".xml"):
        model_path = os.path.join(models_dir, file)
        model_id = os.path.splitext(file)[0]
        model = read_sbml_model(model_path)
        present_rxns = {
            name: rxn_id for name, rxn_id in target_reactions.items()
            if rxn_id in model.reactions
        }

        fva = flux_variability_analysis(model, reaction_list=list(present_rxns.values()))

        row = {"model_id": model_id}
        for name, rxn_id in present_rxns.items():
            row[f"{name}_min"] = fva.loc[rxn_id, "minimum"]
            row[f"{name}_max"] = fva.loc[rxn_id, "maximum"]
        results.append(row)

df = pd.DataFrame(results)
df     

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2658405
Academic license 2658405 - for non-commercial use only - registered to su___@gmail.com
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpf4geuvej.lp
Reading time = 0.00 seconds
: 863 rows, 1729 columns, 7887 nonzeros
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2658405
Academic license 2658405 - for non-commercial use only - registered to su___@gmail.com
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpy1i4fw3l.lp
Reading time = 0.00 seconds
: 863 rows, 1729 columns, 7887 nonzeros
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2658405
Academic license 2658405 - for non-commercial use only - registered to su___@gmail.com
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpo9p4ere_.lp
Reading time = 0.00 seconds
: 863 rows, 1729 

,model_id,Butyrate_min,Butyrate_max,Propionate_min,Propionate_max
0,MAG001.sbml_gapfilled_noO2,0.000000,0.000000,NaN,NaN
1,MAG002.sbml_gapfilled_noO2,NaN,NaN,NaN,NaN
2,MAG003.sbml_gapfilled_noO2,NaN,NaN,NaN,NaN
3,MAG004.sbml_gapfilled_noO2,NaN,NaN,NaN,NaN
4,MAG005.sbml_gapfilled_noO2,NaN,NaN,0.0,0.0
...,...,...,...,...,...
235,MAG236_gapfilled_noO2,0.000000,0.000000,NaN,NaN
236,MAG237_gapfilled_noO2,NaN,NaN,NaN,NaN
237,MAG238_gapfilled_noO2,0.141277,817.876817,NaN,NaN
238,MAG239_gapfilled_noO2,NaN,NaN,NaN,NaN


In [8]:
import pandas as pd

df_fva = df
threshold = 1e-6

def is_blocked(min_val, max_val, threshold):
    if pd.isna(min_val) or pd.isna(max_val):
        return False
    return abs(min_val) < threshold and abs(max_val) < threshold


def classify_row(row):
    butyrate_status = (
        "no_reaction" if pd.isna(row["Butyrate_min"]) or pd.isna(row["Butyrate_max"])
        else "blocked" if is_blocked(row["Butyrate_min"], row["Butyrate_max"], threshold)
        else "active"
    )
    propionate_status = (
        "no_reaction" if pd.isna(row["Propionate_min"]) or pd.isna(row["Propionate_max"])
        else "blocked" if is_blocked(row["Propionate_min"], row["Propionate_max"], threshold)
        else "active"
    )
    return pd.Series([butyrate_status, propionate_status])

df_fva[["Butyrate_status", "Propionate_status"]] = df_fva.apply(classify_row, axis=1)

blocked_butyrate = df_fva[df_fva["Butyrate_status"] == "blocked"]
blocked_propionate = df_fva[df_fva["Propionate_status"] == "blocked"]


print("Blocked Butyrate:", blocked_butyrate.shape[0])
print("Blocked Propionate:", blocked_propionate.shape[0])

Blocked Butyrate: 79
Blocked Propionate: 10


In [11]:
blocked_butyrate.to_csv("../results/blocked_butyrate.csv", index=False)
blocked_butyrate

,model_id,Butyrate_min,Butyrate_max,Propionate_min,Propionate_max,Butyrate_status,Propionate_status
0,MAG001.sbml_gapfilled_noO2,0.0,0.0,NaN,NaN,blocked,no_reaction
12,MAG013.sbml_gapfilled_noO2,0.0,0.0,NaN,NaN,blocked,no_reaction
18,MAG019.sbml_gapfilled_noO2,0.0,0.0,NaN,NaN,blocked,no_reaction
19,MAG020.sbml_gapfilled_noO2,0.0,0.0,NaN,NaN,blocked,no_reaction
23,MAG024.sbml_gapfilled_noO2,0.0,0.0,NaN,NaN,blocked,no_reaction
...,...,...,...,...,...,...,...
229,MAG230_gapfilled_noO2,0.0,0.0,NaN,NaN,blocked,no_reaction
231,MAG232_gapfilled_noO2,0.0,0.0,NaN,NaN,blocked,no_reaction
232,MAG233_gapfilled_noO2,0.0,0.0,NaN,NaN,blocked,no_reaction
235,MAG236_gapfilled_noO2,0.0,0.0,NaN,NaN,blocked,no_reaction


In [13]:
blocked_propionate.to_csv("../results/blocked_propionate.csv", index=False)
blocked_propionate

,model_id,Butyrate_min,Butyrate_max,Propionate_min,Propionate_max,Butyrate_status,Propionate_status
4,MAG005.sbml_gapfilled_noO2,NaN,NaN,0.0,0.0,no_reaction,blocked
9,MAG010.sbml_gapfilled_noO2,NaN,NaN,0.0,0.0,no_reaction,blocked
17,MAG018.sbml_gapfilled_noO2,NaN,NaN,0.0,0.0,no_reaction,blocked
37,MAG038_gapfilled_noO2,NaN,NaN,0.0,0.0,no_reaction,blocked
40,MAG041_gapfilled_noO2,NaN,NaN,0.0,0.0,no_reaction,blocked
69,MAG070_gapfilled_noO2,NaN,NaN,0.0,0.0,no_reaction,blocked
70,MAG071_gapfilled_noO2,0.0,0.0,0.0,0.0,blocked,blocked
98,MAG099_gapfilled_noO2,NaN,NaN,0.0,0.0,no_reaction,blocked
105,MAG106_gapfilled_noO2,NaN,NaN,0.0,0.0,no_reaction,blocked
188,MAG189_gapfilled_noO2,0.0,1000.0,0.0,0.0,active,blocked


In [11]:
model = read_sbml_model("../models/fixed_models/MAG013.sbml_gapfilled_noO2.xml")
butyrate_c = model.metabolites.get_by_id("cpd00211_c0")
butyrate_e = model.metabolites.get_by_id("cpd00211_e0")

transport_reactions = []
for rxn in butyrate_c.reactions:
    if butyrate_e in rxn.metabolites:
        transport_reactions.append(rxn)


for rxn in transport_reactions:
    print(rxn.id, rxn.reaction, f"Bounds: {rxn.lower_bound}, {rxn.upper_bound}")

TR_cpd00211_e0 cpd00211_c0 --> cpd00211_e0 Bounds: 0.0, 1000.0


In [22]:
model.reactions.EX_cpd00211_e0.objective_coefficient = 1
model.reactions.bio1.objective_coefficient = 0
model.optimize()

for rxn in model.reactions:
    if rxn.flux != 0 and model.metabolites.get_by_id("cpd00211_e0") in rxn.metabolites:
        print(rxn.id, rxn.build_reaction_string(use_metabolite_names=1), rxn.flux)

In [41]:
for rxn in model.reactions:
    if model.metabolites.get_by_id("cpd00120_c0") in rxn.metabolites:
        print(rxn.id, rxn.build_reaction_string(use_metabolite_names=1), rxn.flux)

rxn00994_c0 Butyryl-CoA_c0 + Acetoacetate_c0 <=> Butyrate_c0 + Acetoacetyl-CoA_c0 0.0
rxn00872_c0 FAD_c0 + H+_c0 + Butyryl-CoA_c0 <-- Crotonyl-CoA_c0 + FADH2_c0 0.0
rxn00875_c0 Acetate_c0 + Butyryl-CoA_c0 <=> Acetyl-CoA_c0 + Butyrate_c0 0.0
rxn00874_c0 Acetyl-CoA_c0 + Butyryl-CoA_c0 <=> CoA_c0 + 3-Oxohexanoyl-CoA_c0 0.0


In [36]:
model.reactions.rxn00872_c0.build_reaction_string(use_metabolite_names=0)

'cpd00015_c0 + cpd00067_c0 + cpd00120_c0 <-- cpd00650_c0 + cpd00982_c0'

In [30]:
model.reactions.rxn00872_c0.genes

frozenset({<Gene MAG013_1_4 at 0x168744d00>})

In [15]:
## check exchange reactions 
from cobra.io import read_sbml_model


model_dir = "../models/fixed_models/MAG013.sbml_gapfilled_noO2.xml"

butyrate_ids = {"cpd00141_e0", "cpd00141_c0"}
propionate_ids = {"cpd00211_e0", "cpd00211_c0"}

# 初始化计数器
butyrate_models = 0
propionate_models = 0
total_models = 0

# 遍历所有模型文件
for file in os.listdir(model_dir):
    if file.endswith(".xml"):
        filepath = os.path.join(model_dir, file)
        try:
            model = read_sbml_model(filepath)
            total_models += 1

            met_ids = {met.id.lower() for met in model.metabolites}

            # 判断是否含有丁酸和丙酸代谢物
            has_butyrate = any(met in met_ids for met in butyrate_ids)
            has_propionate = any(met in met_ids for met in propionate_ids)

            if has_butyrate:
                butyrate_models += 1
            if has_propionate:
                propionate_models += 1

        except Exception as e:
            print(f"模型读取失败: {file}，错误: {e}")

# 打印统计结果
print(f"总模型数: {total_models}")
print(f"含丁酸 (cpd00141) 的模型数: {butyrate_models}")
print(f"含丙酸 (cpd00211) 的模型数: {propionate_models}")



NotADirectoryError: [Errno 20] Not a directory: '../models/fixed_models/MAG013.sbml_gapfilled_noO2.xml'